# PREDICCIÓN DE LA PRODUCCIÓN DE ENERGÍA EÓLICA CON SCIKIT-LEARN


## Configuración previa

In [32]:
# importacion de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# los modulos de sklearn que se utilizaran se importaran en el momento de su uso

In [33]:
# definicion de constantes usadas a lo largo del proyecto
SEED = 100472050 # la semilla debe ser el NIA de uno de los integrantes
wind_ava = pd.read_csv("data/wind_ava.csv", index_col=0)
wind_comp = pd.read_csv("data/wind_comp.csv", index_col=0)

### Seleccion de el molino 13 (sotavento)

In [34]:
from sklearn.model_selection import train_test_split
# la variable objetivo es la columna energy y solo queremos los datos que
# terminan en 13, agregar la columna energy a wind_ava
aux = wind_ava[wind_ava.columns[wind_ava.columns.str.endswith('13')]]
# añadir la columna energy a wind_ava
aux.insert(0, "energy", wind_ava["energy"])     
wind_ava = aux
del aux

# Primero, dividiremos los datos en entrenamiento y test
X = wind_ava.drop(columns='energy')
y = wind_ava['energy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

### Exploración inicial
 - Estructura.
 - Tipos.
 - Identificación de valores faltantes.

Como se puede observar, haciendo uso de dos modelos de regresión "dummy" se tiene a un error cuadrático medio elevado.
Este modelo lo usaremos para comparar con nuestro modelo real

## Metodo de Escalado con KNN

In [35]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#normalizacion por escala min-max
min_max = make_column_transformer(
    (MinMaxScaler(), X_train.columns)
)

#normalizacion por escala estandar
standard = make_column_transformer(
    (StandardScaler(), X_train.columns)
)


### Escalado min-max

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
import time

pipe_knn = Pipeline([
        ('preproceso', min_max), #entrada del pipeline
        ('regresor', KNeighborsRegressor()) #salida del pipeline
])

np.random.seed(SEED)

t1 = time.time()
pipe_knn.fit(X_train, y_train)
t2 = time.time()

y_test_pred = pipe_knn.predict(X_test)

rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_knn = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_knn}")
print(f"R2: {r2_knn}")
print("Tiempo de entrenamiento: ", t2 - t1)

RMSE: 405.1739421755083
R2: 0.6281266156491352
Tiempo de entrenamiento:  0.007009744644165039


### Escalado estandar

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
import time

pipe_knn = Pipeline([
        ('preproceso', standard), #entrada del pipeline
        ('regresor', KNeighborsRegressor()) #salida del pipeline
])

np.random.seed(SEED)

t1 = time.time()
pipe_knn.fit(X_train, y_train)
t2 = time.time()

y_test_pred = pipe_knn.predict(X_test)

rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_knn = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_knn}")
print(f"R2: {r2_knn}")
print("Tiempo de entrenamiento: ", t2 - t1)

RMSE: 385.2078010641338
R2: 0.6638739052421688
Tiempo de entrenamiento:  0.006002664566040039


El escalado min-max es el que mejor se ajusta a los datos, por lo que lo usaremos en el resto del notebook

## 1.2 puntos A continuación, se considerarán estos métodos: KNN, árboles de regresión, regresión lineal (la normal y al menos, la variante Lasso) y SVM:
a.	Se evaluarán dichos modelos con sus hiperparámetros por omisión. También se medirán los tiempos que tarda el entrenamiento.

b.	Después, se ajustarán los hiperparámetros más importantes de cada método y se obtendrá su evaluación. Medir tiempos del entrenamiento, ahora con HPO.

c.	Obtener algunas conclusiones, tales como: ¿cuál es el mejor método? ¿Cuál de los métodos básicos de aprendizaje automático es más rápido? ¿Los resultados son mejores que los regresores triviales/naive/dummy? ¿El ajuste de hiperparámetros mejora con respecto a los valores por omisión? ¿Hay algún equilibrio entre tiempo de ejecución y mejora de resultados? ¿Es posible extraer de alguna técnica qué atributos son más relevantes? etc.



### Evaluacion de los modelos con los parametros por defecto

#### Random Forest

In [38]:
# ajuste de hiperparametros para random forest basandonos en rmse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

pipe_rf = Pipeline([
        ('preproceso', standard),
        ('regresor', RandomForestRegressor())
])

param_grid = {
    'regresor__n_estimators': [150, 200],
    'regresor__max_depth': [20, None],
    'regresor__min_samples_split': [5, 10],
    'regresor__min_samples_leaf': [2, 4],
    'regresor__bootstrap': [True]
}

grid_search = GridSearchCV(pipe_rf, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
# medir tiempo de entrenamiento
t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()
dt_random_forest_hpo = t2 - t1
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calcular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_rf_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_rf_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_rf_hpo}")
print(f"R2: {r2_rf_hpo}")

KeyboardInterrupt: 

In [ ]:
# Entrenar random forest haciendo una inner cross validation para seleccionar los hiperparametros

from sklearn.model_selection import cross_val_score

# poner rangos amplios para hacer una busqueda exhaustiva
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
print("Buscando hiperparámetros...")

# hacer una busqueda exhaustiva
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
print("Entrenando modelo...")
t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()
dt_rf_cv = t2 - t1

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)
print("Mejor RMSE encontrado (train):")
print(-grid_search.best_score_)
print("Tiempo de entrenamiento:")
print(dt_rf_cv)

# Calcular métricas
best_model = grid_search.best_estimator_

# Predecir los valores de test
y_test_pred = best_model.predict(X_test)

# Calcular métricas
rmse_rf_cv = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_rf_cv = best_model.score(X_test, y_test)

print(f"RMSE: {rmse_rf_cv}")
print(f"R2: {r2_rf_cv}")


In [ ]:
# entrenar con los mejores hiperparametros encontrados y todos los datos
best_model.fit(X, y)

In [ ]:
# exportar el modelo en un archivo pickle
import pickle
print("Exportando modelo...")
with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### Medir la precision del modelo en energy alto y energy bajo

In [ ]:
### Medir la precision del modelo en energy alto y energy bajo
# divid el conjunto de datos en dos, uno con energy alto y otro con energy bajo
# cuando la energía sea menor que el tercer cuantil, se considerará clase “baja”, y 
# cuando sea mayor, clase  “alta”.
low_energy = wind_ava[wind_ava['energy'] < wind_ava['energy'].quantile(1/3)]
high_energy = wind_ava[wind_ava['energy'] > wind_ava['energy'].quantile(2/3)]

X_low = low_energy.drop(columns='energy')
y_low = low_energy['energy']

X_high = high_energy.drop(columns='energy')
y_high = high_energy['energy']

# medir el rendimiento en el conjunto de datos con energía baja
y_low_pred = best_model.predict(X_low)
rmse_low = np.sqrt(metrics.mean_squared_error(y_low, y_low_pred))
r2_low = best_model.score(X_low, y_low)

# medir el rendimiento en el conjunto de datos con energía alta
y_high_pred = best_model.predict(X_high)
rmse_high = np.sqrt(metrics.mean_squared_error(y_high, y_high_pred))
r2_high = best_model.score(X_high, y_high)

print(f"RMSE en energía baja: {rmse_low}")
print(f"R2 en energía baja: {r2_low}")

print(f"RMSE en energía alta: {rmse_high}")
print(f"R2 en energía alta: {r2_high}")


### Convertir el problema en uno de clasificacion
Probar con:
- KNN
- Arboles de clasificacion
- Regresion logistica
- SVM
- Random Forest
- XGBoost
- LightGBM

In [ ]:
# ETIQUETAR LOS DATOS COMO LOW Y HIGH energy
# añadir una columna al conjunto de datos original que indique si la energía es baja o alta
wind_ava['energy_class'] = 'high'
wind_ava.loc[wind_ava['energy'] < wind_ava['energy'].quantile(1/3), 'energy_class'] = 'low'
# eliminar la columna energy
wind_ava.drop(columns='energy', inplace=True)

wind_ava.head()


#### KNN